# Implementation and evaluation of a computational standardization pipeline for chemical compounds
## Based on ["Trust, But Verify: On the Importance of Chemical Structure Curation in Cheminformatics and QSAR Modeling Research" from 2010 (D. Fourches, ...)"](https://pubmed.ncbi.nlm.nih.gov/20572635/)


### Introduction 
This notebook serves to showcase the functionality of the Standardization module. Following the recommended standardization steps of "Trust, But Verify"(D. Fourches, ..., 2010).
This notebook uses the dataset of following paper: [Cheminformatics Analysis of Assertions Mined from Literature That Describe Drug-Induced Liver Injury in Different Species](https://pubs.acs.org/doi/10.1021/tx900326k)

<span style='color:red'> For all relative paths in this notebook to work, please make sure you are starting this notebook from the working directory ./opencadd/docs/tutorials/  </span><br>
Check your directory with the cell below.

In [5]:
import os
os.getcwd()

'/home/allen/dev/opencadd/docs/tutorials'

In [1]:
#Import pandas and numpy
import pandas as pd
import numpy as np

#import modules and Standardization API functions needed
from rdkit import Chem
from opencadd.compounds.standardization import convert_format,handle_fragments,disconnect_metals,detect_inorganic,remove_salts

In [2]:
# Helper function to mark at which step the entry failed the standardization pipeline
def failMarker(i):
    i=stepNo
    return i

###  Step 0: Initial dataset import and cleaning
------------------------------------------------
The first step before the standardization steps are started is a import of the dataset as an Pandas Dataframe, only including the columns necessary. In this case we use the <b>Names</b> and <b>SMILEs</b> column.<br>
Then we search for all entries which actually don't have any strings saved under <b>SMILEs</b> and kick them from the dataset, since they are not holding any information.<br>
After the import we add a <b>Failed_at</b> column to track in which standardization step the entry failed. 
The intial `stepNo` will be 0, which leads to an default <b>Failed_at</b>-value of 0 for all entries. The <b>Failed_at</b>-value will be used to filter out all failed entries for the upcoming steps.

In [51]:
stepNo = 0

# Importing the test-dataset 
dataset = pd.read_csv (r'./data/standardization_test_data.csv')

#Filter for needed columns
dataset = dataset[["Names","SMILEs"]]

#Kick all empty entries
dataset = dataset[(dataset['SMILEs'].notna())]

#Setting a initial score of 0 for all entries
dataset['Failed_at'] = dataset['SMILEs'].notna().apply(failMarker)

#Show the current form of the dataframe
dataset.head()

,Names,SMILEs,Failed_at
0,(R)-Roscovitine,CCC(CO)Nc1nc(NCc2ccccc2)c2ncn(C(C)C)c2n1,0
1,17-Methyltestosterone,CC1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3CCC12C,0
2,1-alpha-Hydroxycholecalciferol,CC(C)CCCC(C)C1CCC2C(CCCC12C)=CC=C1CC(O)CC(O)C1=C,0
3,"2,3-Dimercaptosuccinic acid",OC(=O)C(S)C(S)C(O)=O,0
4,"2,4,6-Trinitrotoluene",Cc1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,0


### Step 1: Conversion of SMILEs to mol
------------------------------------------
### Convert the SMILE representation format of the compounds into Mol-files

RDKit performs a sanitization of the molecule by default. In this sanitization step RDKit tries to kekulize the mols (generates alternate Lewis structures). This step might fail, when the structure is aromatic, but no Hydrogen position is provided. TODO:!(This explanation might be a bit short and not fully correct, check this later again)!

If the conversion from SMILE to mol fails, then those SMILEs will get a <b>Failed_at</b> marker added. 

To avoid the sanitization of the molecule `convert_smiles_to_mol` can be called with the argument `sanitize=False`. Keep in mind that the generation of different Lewis structures serves to find different representation formats of the same molecule. 

References:<br>
https://chemistry.stackexchange.com/questions/116498/what-is-kekulization-in-rdkit<br>
https://rdkit-discuss.narkive.com/QwnqcKcM/another-can-t-kekulize-mol-observation<br>
https://www.rdkit.org/docs/Cookbook.html<br>
https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html<br>


In [56]:
# We set up the stepNo and create a subset with only the entries which haven't failed yet.
stepNo = 1
work_dataset =  dataset[(dataset['Failed_at']==0)]

# A column called mol is beeing added to the dataframe to store the mol-files
dataset['mol'] = work_dataset['SMILEs'].apply(convert_format.convert_smiles_to_mol)

# Checking for SMILEs, where no mol could be generated. Marking them with the failMarker
dataset['Failed_at'] = np.where(dataset['mol'].isnull(),dataset['Failed_at'].apply(failMarker),dataset['Failed_at'] )

# See how the entry in row 14 has a Failed_at value of 1.
dataset.head(16)


,Names,SMILEs,Failed_at,mol
0,(R)-Roscovitine,CCC(CO)Nc1nc(NCc2ccccc2)c2ncn(C(C)C)c2n1,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d1877b0>
1,17-Methyltestosterone,CC1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3CCC12C,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187990>
2,1-alpha-Hydroxycholecalciferol,CC(C)CCCC(C)C1CCC2C(CCCC12C)=CC=C1CC(O)CC(O)C1=C,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187760>
3,"2,3-Dimercaptosuccinic acid",OC(=O)C(S)C(S)C(O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187a30>
4,"2,4,6-Trinitrotoluene",Cc1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187940>
5,2-Deoxy-D-glucose,OCC1OC(O)CC(O)C1O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d1879e0>
6,2'-fluoro-5-methylarabinosyluracil,CC1=CN(C2OC(CO)C(O)C2F)C(=O)NC1=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187ad0>
7,2-Methoxyestradiol,COc1cc2C3CCC4(C)C(O)CCC4C3CCc2cc1O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187a80>
8,4-aminobenzoic acid,Nc1ccc(cc1)C(O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187b20>
9,4-Hydroxytamoxifen,CCC(c1ccccc1)=C(c1ccc(O)cc1)c1ccc(OCCN(C)C)cc1,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187bc0>


### Step 2: Removal of Inorganics and Mixtures
--------------------------------------------------

Since molecular descriptors can only be computed for organic compunds, all inorganic compunds must be removed before the descriptors are calculated. (Chapter 2.1. Fourches 2010)

For the flagging and following removal of compounds containing inorganic molecules, we can use the function `detect_inorganic`. This function returns a boolean value of "True" when it finds a inorganic molecule. We can run this flagging in a pre-processing step of the data, and discard those compounds. 
"Inorganic compounds are known to have biological effects, like for example toxic effects."(Chapter 2.1. Fourches 2010)(fix citation)
 Due to their potential bioactivity we can not distinguish if the recored activity of a mixed compound is caused by it's organic or inorganic part. Therefore the entry is useless and can be discarded. ! THIS SHOULD BE LOGGED AND MANUAL CURATION SHOULD BE ENABLED !
An alternate and easy way would be that every SMILES  is undertaken a substring search, where a match of a inorganic compound pattern (search pattern set should be defined) would be  flagged.

Due to the fact, that the treatment is not as simple as it apprears the paper (Fourches, 2010) recommends to delete records containing mixtures. ! THIS AGAIN CAN BE LOGGED AND MANUAL CURATION CAN BE DONE WITH THIS SET ! The ease up the curation various filtering functions can be implemented to help decide which to keep and which to discard. Three types of mixtures are described. ! CHECK IF IMPLEMENTATION WOULD BE POSSIBLE EASY AND FAST ! Common and widely used practice is to retain molecules with the highest molecular weight or the largest number of atoms(Chapter 2.1. Fourches 2010), but the paper (Fourches, 2010) states this might not be the best solution, and further investigation in mixtures should only be done if there is a reason to belive that the biological activity is really caused by the largest molecule and not the mixture itself.

Those actions might be performed, before the entered SMILES are beeing converted into mol-files. Some described steps are related to string pattern searches.

In [57]:
# We set up the stepNo and create a subset with only the entries which haven't failed yet.
stepNo = 2
work_dataset =  dataset[(dataset['Failed_at']==0)]
work_dataset.head(16)

,Names,SMILEs,Failed_at,mol
0,(R)-Roscovitine,CCC(CO)Nc1nc(NCc2ccccc2)c2ncn(C(C)C)c2n1,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d1877b0>
1,17-Methyltestosterone,CC1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3CCC12C,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187990>
2,1-alpha-Hydroxycholecalciferol,CC(C)CCCC(C)C1CCC2C(CCCC12C)=CC=C1CC(O)CC(O)C1=C,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187760>
3,"2,3-Dimercaptosuccinic acid",OC(=O)C(S)C(S)C(O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187a30>
4,"2,4,6-Trinitrotoluene",Cc1c(cc(cc1N(=O)=O)N(=O)=O)N(=O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187940>
5,2-Deoxy-D-glucose,OCC1OC(O)CC(O)C1O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d1879e0>
6,2'-fluoro-5-methylarabinosyluracil,CC1=CN(C2OC(CO)C(O)C2F)C(=O)NC1=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187ad0>
7,2-Methoxyestradiol,COc1cc2C3CCC4(C)C(O)CCC4C3CCc2cc1O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187a80>
8,4-aminobenzoic acid,Nc1ccc(cc1)C(O)=O,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187b20>
9,4-Hydroxytamoxifen,CCC(c1ccccc1)=C(c1ccc(O)cc1)c1ccc(OCCN(C)C)cc1,0,<rdkit.Chem.rdchem.Mol object at 0x7f1d8d187bc0>


In [ ]:
#  Removal of mixtures, inorganics (and eventually organometallics)
# Functions detect_inorganic,remove_fragments, disconnect_metals, detect_inorganic again

In [ ]:
# Pseudocode for filtering the inorganic records

new_data = array of smiles
records_organics = []
records_inorganics = []

for x in new_data:
    convert_smiles_to_mol(x)
    if detect_inorganic(x)=="False":
        records_organics.append(x) #QUESTION: Can I just store mol-files in an array?
    elif detect_inorganic(x)=="True":
        records_inorganics.append(x)
    else:
        raise Exception("Something is wrong with:" x)
return records_organics, records_inorganics
# no further processing will happen to `records_inorganics`
# `records_organics` is passed on in the pipeline


In [ ]:
# Removal of inorganics and mixtures of the dataset
dataset['Inorganics'] = dataset['mol'].apply(detect_inorganic)
dataset.to_csv('/home/allen/dev/utility/data/test_dataset_bool_inorganic.csv', index = False)
contains_inorganics = dataset[dataset['Inorganics']== True]
contains_inorganics.head()
dataset = dataset[dataset['Inorganics']== False]
dataset = dataset[["Names","SMILEs","mol"]]
dataset.head()

In [ ]:
# Flagging of Mixtures
dataset['InchIBeforeMixturesFiltering'] = dataset['mol'].apply(convert_format.convert_mol_to_inchi)
dataset['MolafterMixturesFiltering'] = dataset['mol'].apply(handle_fragments.remove_fragments)
dataset['InchIAfterMixturesFiltering'] = dataset['MolafterMixturesFiltering'].apply(convert_format.convert_mol_to_inchi)
dataset['noChanges']= dataset['InchIBeforeMixturesFiltering'] == dataset['InchIAfterMixturesFiltering']
dataset.to_csv('/home/allen/dev/utility/data/test_dataset_bool_mixtures.csv', index = False)
dataset.head()

#TODO: Subset of Mixtures, deletion from main set

In [ ]:
# Pseudocode for filtering the mixture records

new_data = records_organics
no_fragement_records = []
contains_fragment_records = []

#TODO: Write a helper function that returns a boolean value, when it finds a fragment (the InChI is changed by a removed fragment), a metals (disconnect_metals has been performed)
# Functions could be named `detect_fragment` and  `detect_metals`
# Or I can write a function that just checks if the execution of a function actually altered the InChI --> might be the simpler solution
for x in new_data:
    convert_smiles_to_mol(x)
    if detect_fragment(x)=="False":
        no_fragement_record.append(x)
    elif detect_fragment(x)=="True":
        contains_fragment_records.append(x)
    else:
        raise Exception("Something is wrong with" x)
return no_fragement_record,contains_fragment_records

# [OPTIONAL] return the largest fragment of the record
new_data = contains_fragment_records
contains_largest_fragement = []
for x in new_data:
    choose_largest_fragment(x)
    contains_largest_fragement.append(x)
return contains_largest_fragement

# OR remove known common fragments with the function `remove_fragments`

# Code would be the same as obove
# Have to write a helper function to continue the pipeline with his set, without mixing it up with the "safe_dataset"






In [ ]:
# Pseudocode for filtering metals 
new_data = no_fragement_record
metal_true = []
metal_false = []

for x in new_data:
    convert_smiles_to_mol(x)
    if detect_metals(x)=="False":
        metal_false.append(x)
    elif detect_metals(x)=="True":
        metal_true.append(x)
    else:
        raise Exception("Something is wrong with" x)
return metal_true,metal_false

### Structural Conversion and Cleaning

Some drugs need to be transformed "into their salt form to enhance how the drug disscolves (...) and (to) increase it's effectiveness. (https://www.drugs.com/article/pharmaceutical-salts.html (03/12/21)) Therefore it is common for chemical compound databases to contain records of salts. If possible it is recommended to delete the records containing salts completely, since, similar to in-organic compounds, "most descriptor-generating software (can not process salts)" (Fourches 2010 Chapter 2.2 ).While not beeing desirable, it is still an acceptable procedure to convert compounds into their neutral forms. But cases like this should be tagged, filtered and afterwards manually curated or compared to the actual neutral form of that compound. 
In case that we want to continue working on the converted records, we should perform the following steps:
- check if records contain compounds with presence of metals --> difficult case, filter out (already done this - one step ahead)
- removing the salts from the record
- neutralize the record (normalization or basic standardization)
- neutralize the charges
- to be discussed: the adding/removing of hydrogens, both got pros and cons (pro addingH --> higher prediction performances / con addingH --> may introduce noise --> less reliable models)(removingH might introduce erros in calculating descriptors, due to it might not handle certain cases well)



In [ ]:
# Structural coversion
# Cleaning/removal of salts
# Functions remove_salts
# normalize_molecules
# handle_charges
# handle_hydrogens

### Normalization of Specific Chemotypes

More complex than just Normalization.

In [ ]:
# Normalization of specific chemotypes
# normalize_molecules

In [ ]:
# Treatment of tautomeric forms
# handle_tautomers

### Removal of duplicates

In [ ]:
# Analysis/removal of duplicates

In [ ]:
# Manual inspection